colab에서 학습한 ssd 모델을 이용해서 local에서 hand sign letter들을 추론
- Tensorflow 설치
- Tensorflow Object Detection API2 설치
    - 설치, 환경변수(Pythonpath) 설정
- pyyaml 라이브러리 설치
    - pip install pyyaml
- 학습된 모델
    - checkpoint 파일
    - labelmap 설정파일
    - pipeline.config 설정파일

In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import temsorflow as tf
from object_detection.utils import label_map_util, config_util
from object_detection.utils import visualization as viz_utils
from object_detection.builders import model_builder

ModuleNotFoundError: No module named 'cv2'

## 경로를 변수로 저장

In [ ]:
PIPELINE_CONFIG_PATH = 'config/pipeline.config'
LABELMAP_PATH = 'config/label_map.pdtxt'
CHECKPOINT_DIR_PATH = 'checkpoint'

## 모델생성(Restore)

In [3]:
config = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_PATH)   # pipeline config 파일 읽어오기
detection_model = model_builder.build(model_config=config['model'], is_training=False)  # pipeline config 설정에 맞춰 모델을 생성

# 모델에 학습된checkpoint(weight저장)를 주입
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_DIR_PATH, 'ckpt-51')).expect_partial()

#detection_model: 학습된 SSD 모델을 복원

NameError: name 'config_util' is not defined

### detection 함수
-  tf.Tensor를 입력 이미지로 받는다.
- data 전처리 -> 추론 -> 후처리

In [ ]:
def detect_funtion(image):
    image, shapes = detection_model.preprocess(image)  # resize, normalize -> (전처리된 img, img shape)
    pred = detection_model.predict(image, shape)  # 모든 bbox를 추론
    result = detection_model.postprocess(pred, shapes)  # confidence score 높은 100개의 bbox만 추출
    
    return result